# Create a simple recommendation system that suggests items tousers based on their preferences. You can use techniques likecollaborative filtering or content-based filtering to recommendmovies, books, or products to users.

# 1.Import Libraries

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## 2. Load Data

In [4]:
data = {
    'user_id': [1, 1, 1, 2, 2, 3, 3, 3, 3, 4, 4, 5, 5],
    'movie_id': [1, 2, 3, 1, 4, 2, 3, 4, 5, 2, 3, 4, 5],
    'rating': [4, 5, 3, 5, 2, 4, 3, 4, 2, 4, 5, 3, 4]
}

ratings_df = pd.DataFrame(data)

## 3. Create a User-Item Matrix

In [5]:
user_item_matrix = ratings_df.pivot_table(index='user_id', columns='movie_id', values='rating').fillna(0)

## 4. Compute Similarity Matrix

In [6]:
user_similarity = cosine_similarity(user_item_matrix)
np.fill_diagonal(user_similarity, 0)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

## 5. Predict Ratings

In [7]:
def predict_ratings(user_item_matrix, user_similarity_df):
    mean_user_rating = user_item_matrix.mean(axis=1)
    ratings_diff = (user_item_matrix - mean_user_rating[:, np.newaxis])
    pred = mean_user_rating[:, np.newaxis] + user_similarity_df.dot(ratings_diff) / np.array([np.abs(user_similarity_df).sum(axis=1)]).T
    return pred

predicted_ratings = predict_ratings(user_item_matrix.values, user_similarity)
predicted_ratings_df = pd.DataFrame(predicted_ratings, index=user_item_matrix.index, columns=user_item_matrix.columns)

## 6. Make Recommendations


In [8]:
def recommend_movies(user_id, num_recommendations):
    sorted_user_predictions = predicted_ratings_df.loc[user_id].sort_values(ascending=False)
    user_rated_movies = user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] > 0].index.tolist()
    recommended_movies = sorted_user_predictions.index[~sorted_user_predictions.index.isin(user_rated_movies)]
    return recommended_movies[:num_recommendations]

user_id = 1
num_recommendations = 3
recommendations = recommend_movies(user_id, num_recommendations)
print(f"Recommended movies for user {user_id}: {recommendations.tolist()}")

Recommended movies for user 1: [4, 5]


### Content-Based Filtering

## 1. Sample Data

In [9]:
books_data = {
    'book_id': [1, 2, 3, 4, 5],
    'title': ['Book A', 'Book B', 'Book C', 'Book D', 'Book E'],
    'genre': ['Fantasy', 'Sci-Fi', 'Mystery', 'Fantasy', 'Sci-Fi'],
    'author': ['Author 1', 'Author 2', 'Author 3', 'Author 1', 'Author 2']
}

books_df = pd.DataFrame(books_data)

## 2. Feature Engineering

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
books_df['features'] = books_df['genre'] + ' ' + books_df['author']
count_vectorizer = CountVectorizer()
features_matrix = count_vectorizer.fit_transform(books_df['features'])

## 3. Compute Similarity Matrix

In [11]:
book_similarity = cosine_similarity(features_matrix)
book_similarity_df = pd.DataFrame(book_similarity, index=books_df['title'], columns=books_df['title'])

## 4. Recommend Books

In [12]:
def recommend_books(book_title, num_recommendations):
    similar_books = book_similarity_df[book_title].sort_values(ascending=False)
    similar_books = similar_books.drop(book_title)
    return similar_books.index[:num_recommendations]

book_title = 'Book A'
num_recommendations = 2
recommended_books = recommend_books(book_title, num_recommendations)
print(f"Books similar to '{book_title}': {recommended_books.tolist()}")

Books similar to 'Book A': ['Book D', 'Book C']
